<a href="https://colab.research.google.com/github/imanuni/imanuni/blob/main/model-de-chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from transformers import BertTokenizer, BertForTokenClassification, BertConfig, Trainer, TrainingArguments
from transformers import DataCollatorForTokenClassification
from datasets import load_dataset, load_metric, DatasetDict

# Charger et préparer les données
dataset = load_dataset("conll2003")  # Remplacez par votre propre jeu de données annoté

# Prétraitement
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

ModuleNotFoundError: No module named 'datasets'

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)


In [ ]:

# Chargement du modèle BERT
model = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(dataset['train'].features['ner_tags'].feature.names))

# Préparer l'entraînement
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [ ]:

data_collator = DataCollatorForTokenClassification(tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Entraîner le modèle
trainer.train()

# Évaluation du modèle
metric = load_metric("seqeval")

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

trainer.evaluate()

# Sauvegarde du modèle
model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")

Assurez-vous d'avoir installé les bibliothèques nécessaires avant d'exécuter le code :

In [ ]:
pip install transformers datasets seqeval

donne moi un examplaire des resultats

In [ ]:
import torch
from transformers import BertTokenizer, BertForTokenClassification
import numpy as np

# Charger le modèle et le tokenizer sauvegardés
model = BertForTokenClassification.from_pretrained('./saved_model')
tokenizer = BertTokenizer.from_pretrained('./saved_model')

# Exemples de textes journalistiques
texts = [
    "Le président a déclaré que les nouvelles mesures économiques seront bénéfiques pour le pays.",
    "Le ministre des Affaires étrangères a annoncé une coopération renforcée avec les pays voisins."
]

# Fonction pour prédire et afficher les résultats
def predict_and_display(texts):
    for text in texts:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, is_split_into_words=True)
        with torch.no_grad():
            outputs = model(**inputs)

        predictions = torch.argmax(outputs.logits, dim=2)
        tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

        print(f"Texte : {text}")
        print("Déclarations politiques extraites :")

        current_declaration = []
        for token, label in zip(tokens, predictions[0].numpy()):
            if label == 1:  # Supposons que l'étiquette 1 correspond aux déclarations politiques
                current_declaration.append(token)
            else:
                if current_declaration:
                    print(" ".join(current_declaration).replace(' ##', ''))
                    current_declaration = []

        if current_declaration:
            print(" ".join(current_declaration).replace(' ##', ''))
        print("\n")

# Prédire et afficher les résultats pour les textes de test
predict_and_display(texts)

Résultats attendus :
En exécutant le code ci-dessus, vous obtiendrez une sortie similaire à celle-ci (les résultats réels dépendront des performances du modèle entraîné) :
Texte : Le président a déclaré que les nouvelles mesures économiques seront bénéfiques pour le pays.
Déclarations politiques extraites :
Le président a déclaré

Texte : Le ministre des Affaires étrangères a annoncé une coopération renforcée avec les pays voisins.
Déclarations politiques extraites :
Le ministre des Affaires étrangères a annoncé